In [2]:
from pyspark.sql import SparkSession


In [3]:
spark = SparkSession.builder.appName("rdd").getOrCreate()
spark

In [5]:
my_range = spark.range(100).toDF("number")
my_range.show()


+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows



In [8]:
my_range.where("number %2 ==0").show()

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
|    10|
|    12|
|    14|
|    16|
|    18|
|    20|
|    22|
|    24|
|    26|
|    28|
|    30|
|    32|
|    34|
|    36|
|    38|
+------+
only showing top 20 rows



In [11]:
divis_by_2 = my_range.where("number%2 = 0").take(3)


0

In [12]:
# end to end example 
flight_data_2015 = spark.read\
                    .option("inferSchema",True)\
                    .option("header",True)\
                    .csv("data/flight-data/csv/2015-summary.csv")

flight_data_2015

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]

In [14]:
flight_data_2015.show(4)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
+-----------------+-------------------+-----+
only showing top 4 rows



In [13]:
flight_data_2015.createOrReplaceTempView("flight_data_15")

In [11]:
flight_data_2015.createOrReplaceTempView("flight_data_2015")

In [25]:
# find toatal flights per country
from pyspark.sql import functions as F 
flight_data_2015\
    .groupBy("DEST_COUNTRY_NAME").agg(F.avg("count").alias("average"),F.sum("count").alias("total"))\
    .orderBy(F.col("total").desc()).show()

# using sql
spark.sql("select DEST_COUNTRY_NAME, avg(count) as average , sum(count) as total from flight_data_15 group by DEST_COUNTRY_NAME order by total desc").show()

+------------------+--------+------+
| DEST_COUNTRY_NAME| average| total|
+------------------+--------+------+
|     United States|3290.816|411352|
|            Canada|  8399.0|  8399|
|            Mexico|  7140.0|  7140|
|    United Kingdom|  2025.0|  2025|
|             Japan|  1548.0|  1548|
|           Germany|  1468.0|  1468|
|Dominican Republic|  1353.0|  1353|
|       South Korea|  1048.0|  1048|
|       The Bahamas|   955.0|   955|
|            France|   935.0|   935|
|          Colombia|   873.0|   873|
|            Brazil|   853.0|   853|
|       Netherlands|   776.0|   776|
|             China|   772.0|   772|
|           Jamaica|   666.0|   666|
|        Costa Rica|   588.0|   588|
|       El Salvador|   561.0|   561|
|            Panama|   510.0|   510|
|              Cuba|   466.0|   466|
|             Spain|   420.0|   420|
+------------------+--------+------+
only showing top 20 rows

+------------------+--------+------+
| DEST_COUNTRY_NAME| average| total|
+-----------

In [12]:
from pyspark.sql import functions as F 

flight_data_2015\
    .groupBy("DEST_COUNTRY_NAME")\
    .sum("count")\
    .withColumnRenamed("sum(count)","destination_total")\
    .sort(F.desc("destination_total"))\
    .limit(5)\
    .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [3]:
spark = SparkSession.builder.appName("rdd").getOrCreate()

# lower level api 
- rdd 
- accumulatorw
- broadcast

# rdd 


In [31]:
collection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")

words = spark.sparkContext.parallelize(collection,2)

In [32]:
words.getNumPartitions()

2

In [33]:
words.collect()

['Spark',
 'The',
 'Definitive',
 'Guide',
 ':',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

In [34]:
words.setName("mywords")

mywords ParallelCollectionRDD[115] at readRDDFromFile at PythonRDD.scala:262

In [35]:
words.name

<bound method RDD.name of mywords ParallelCollectionRDD[115] at readRDDFromFile at PythonRDD.scala:262>

In [9]:
words.take(2)

['Spark', 'The']

## transoformation
1. distinct 
2. map 
3. filter 
4. flatmap 
5. sort 
6. random split 

## action 
1. reduce 
2. count / countApprox
3. countByValue
4. first
5. max and min 
6. take
7.  

## saving files 
1. saveAsTextFile
2. sequence file saveAsObjectFile
3. 

## caching
wrods.cache()

# checkpointing

##  partition
1. mapPartition
2. foreachPartition
3. glom 

In [38]:

words = spark.sparkContext.parallelize(collection,4)
words.glom().collect()

[['Spark', 'The'],
 ['Definitive', 'Guide'],
 [':', 'Big'],
 ['Data', 'Processing', 'Made', 'Simple']]

In [13]:
spark.stop()